<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d4a99895d0da693256f9e7b72cd7cb4b47999f2a95284e2d58c764c61b733580
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 13:45:05
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -19.77 K (-0.14%)
Current PnL: -21.82 L (-14.3%)
CY Booked + Current PnL: -9.17 L (-6.01%)
-------------------
Total profit:  2.02 L
Total loss:  -23.84 L
-------------------
Total Booked + Current PnL: 17.81 L (14.18%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.86%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.81 L (62.21%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,63.0,X-LC,6.44,231691.0,21325.0,7460.0,-0.73,10.14,3.22,13.68,37.0,2.86,1.64,28.34,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,68.0,M-LC,11.82,174018.0,33178.0,11311.0,1.93,23.56,6.50,31.59,66.0,2.93,1.23,56.29,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.42,53.0,M-SC,1.94,87619.0,-13158.0,13248.0,0.83,-13.06,15.12,0.09,245.0,-0.99,0.62,15.11,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.14,59.0,X-MC,5.87,149487.0,13354.0,17236.0,0.95,9.81,11.53,22.47,91.0,0.77,1.06,24.06,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.87,35.0,X-MC,2.86,87150.0,-3528.0,18432.0,-1.37,-3.89,21.15,16.44,101.0,-0.19,0.62,15.50,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.0,18.88,63.0,X-SC,6.03,210330.0,19324.0,38385.0,3.16,10.12,18.25,30.21,157.0,0.50,1.48,33.11,X40N,NTT,FINANCE
84,VOLTAS,1530.0,-5.41,37.0,X-MC,4.41,198510.0,6768.0,30987.0,2.40,3.53,15.61,19.69,99.0,0.22,1.40,10.57,XY25,NTT,AC
61,SAIL,228.0,53.97,66.0,M-MC,10.42,242991.0,18029.0,149634.0,2.32,8.01,61.58,74.53,192.0,0.12,1.71,42.85,XY24,BTT,STEEL
79,UNIONBANK,163.0,-3.88,68.0,M-LC,11.82,174018.0,33178.0,11311.0,1.93,23.56,6.50,31.59,66.0,2.93,1.23,56.29,XY24,NTT,BANKS
34,ICICIPRULI,790.0,-19.39,60.0,X-MC,1.21,138741.0,2953.0,39805.0,1.68,2.18,28.69,31.48,107.0,0.07,0.98,14.56,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SYMPHONY,1306.0,-44.94,42.0,M-SC,16.74,117166.0,-53975.0,53920.0,-4.72,-31.54,46.02,-0.03,196.0,-1.00,0.83,2.15,OX40N,NTT,DURABLES
1,ABB,7934.0,-42.40,35.0,H-LC,5.09,240720.0,-20899.0,140123.0,-4.07,-7.99,58.21,45.57,7.0,-0.15,1.70,2.43,AR,NTT,ELECTRICAL
70,SURYODAY,240.0,60.09,46.0,H-SC,14.96,146880.0,-32191.0,97925.0,-3.45,-17.98,66.67,36.71,135.0,-0.33,1.04,45.73,XR,NTT,BANKS
17,CERA,9475.0,-24.34,42.0,H-SC,1.85,137850.0,-38053.0,80077.0,-3.09,-21.63,58.09,23.89,149.0,-0.48,0.97,19.05,OX40N,NTT,CERAMICS
19,COFFEEDAY,80.0,-49.70,32.0,L-SC,27.47,73075.0,-40474.0,80521.0,-2.93,-35.64,110.19,35.27,268.0,-0.50,0.52,78.85,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.95,51.0,H-SC,7.95,126101.0,804.0,58120.0,-2.03,0.64,46.09,47.03,141.0,0.01,0.89,57.89,MH,ATH,POWER
56,RECLTD,446.00,42.86,47.0,M-LC,4.72,202235.0,-935.0,43056.0,1.38,-0.46,21.29,20.74,55.0,-0.02,1.43,6.37,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,47.0,M-LC,9.80,206190.0,1114.0,111446.0,1.46,0.54,54.05,54.89,52.0,0.01,1.46,24.77,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.42,53.0,M-SC,1.94,87619.0,-13158.0,13248.0,0.83,-13.06,15.12,0.09,245.0,-0.99,0.62,15.11,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.75,221400.0,-48267.0,84597.0,-0.88,-17.90,38.21,13.47,138.0,-0.57,1.56,12.58,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,42.0,H-SC,1.85,137850.0,-38053.0,80077.0,-3.09,-21.63,58.09,23.89,149.0,-0.48,0.97,19.05,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19943.48,63.0,M-SC,1.51,97972.0,-29833.0,29823.0,0.09,-23.34,30.44,-0.01,232.0,-1.00,0.69,10.94,OX40N,NTT,AC
67,SIS,528.0,1990.67,42.0,H-SC,2.53,84380.0,-26652.0,50257.0,-2.50,-24.00,59.56,21.26,156.0,-0.53,0.60,14.01,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,68.0,M-LC,11.82,174018.0,33178.0,11311.0,1.93,23.56,6.5,31.59,66.0,2.93,1.23,56.29,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,53.97,66.0,M-MC,10.42,242991.0,18029.0,149634.0,2.32,8.01,61.58,74.53,192.0,0.12,1.71,42.85,XY24,BTT,STEEL
31,HINDZINC,730.22,26.60,47.0,M-LC,9.80,206190.0,1114.0,111446.0,1.46,0.54,54.05,54.89,52.0,0.01,1.46,24.77,X5K,ATH,METALS
46,KPIGREEN,731.05,5.95,51.0,H-SC,7.95,126101.0,804.0,58120.0,-2.03,0.64,46.09,47.03,141.0,0.01,0.89,57.89,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,68.0,M-LC,11.82,174018.0,33178.0,11311.0,1.93,23.56,6.50,31.59,66.0,2.93,1.23,56.29,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,6.25,123630.0,8832.0,85206.0,-0.47,7.69,68.92,81.92,223.0,0.10,0.87,46.36,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,6.25,123630.0,8832.0,85206.0,-0.47,7.69,68.92,81.92,223.0,0.10,0.87,46.36,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129447.0,6111.0,125512.0,-1.24,4.95,96.96,106.72,119.0,0.05,0.91,29.49,AR,ATH,MISC
46,KPIGREEN,731.05,5.95,51.0,H-SC,7.95,126101.0,804.0,58120.0,-2.03,0.64,46.09,47.03,141.0,0.01,0.89,57.89,MH,ATH,POWER
31,HINDZINC,730.22,26.60,47.0,M-LC,9.80,206190.0,1114.0,111446.0,1.46,0.54,54.05,54.89,52.0,0.01,1.46,24.77,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,39.0,M-LC,5.42,146909.0,-4036.0,113487.0,-1.29,-2.67,77.25,72.51,53.0,-0.04,1.04,3.19,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.04,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87308.0,-41362.0,86662.0,0.11,-32.15,99.26,35.20,219.0,-0.48,0.62,0.21,X40,NTT,FOOTWEAR
23,DMART,5391.45,-21.15,25.0,X-LC,7.55,200865.0,-9542.0,68716.0,-1.59,-4.54,34.21,28.12,38.0,-0.14,1.42,18.02,X40N,ATH,FMCG
44,JSWINFRA,342.00,-24.37,26.0,X-MC,4.54,189979.0,-10368.0,40864.0,-0.86,-5.17,21.51,15.23,178.0,-0.25,1.34,25.41,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-13.94,32.0,X-LC,6.41,241570.0,-15493.0,50633.0,-0.05,-6.03,20.96,13.67,24.0,-0.31,1.70,14.04,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.34,37.0,X-MC,0.21,198870.0,-1950.0,69744.0,0.73,-0.97,35.07,33.76,80.0,-0.03,1.40,4.18,X40,ATH,FMCG
41,ITC,452.00,-38.77,42.0,X-LC,1.07,197898.0,-2240.0,23134.0,-0.69,-1.12,11.69,10.44,4.0,-0.10,1.40,4.38,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.18,38.0,H-LC,1.39,152420.0,-33675.0,81148.0,-1.14,-18.10,53.24,25.51,15.0,-0.41,1.08,12.32,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.45,38.0,X-MC,3.20,305550.0,-22441.0,128973.0,0.88,-6.84,42.21,32.48,92.0,-0.17,2.16,3.80,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.24,64.0,X-LC,3.96,245908.0,11764.0,36862.0,0.93,5.02,14.99,20.77,86.0,0.32,1.74,12.29,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87308.0,-41362.0,86662.0,0.11,-32.15,99.26,35.20,219.0,-0.48,0.62,0.21,X40,NTT,FOOTWEAR
51,PAGEIND,51769.92,-29.85,32.0,X-MC,7.42,79320.0,-4190.0,24216.0,-0.25,-5.02,30.53,23.98,82.0,-0.17,0.56,0.51,X40,ATH,APPARELS
53,QUESS,986.00,-49.86,36.0,X-SC,37.39,49604.0,-15402.0,166332.0,1.52,-23.69,335.32,232.18,198.0,-0.09,0.35,1.52,XY24,NTT,MISC
20,COLPAL,3726.84,-7.18,40.0,X-MC,7.28,216740.0,-46625.0,155944.0,-0.29,-17.70,71.95,41.51,84.0,-0.30,1.53,1.71,XY25,ATH,FMCG
1,ABB,7934.00,-42.40,35.0,H-LC,5.09,240720.0,-20899.0,140123.0,-4.07,-7.99,58.21,45.57,7.0,-0.15,1.70,2.43,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,36.0,X-SC,37.39,49604.0,-15402.0,166332.0,1.52,-23.69,335.32,232.18,198.0,-0.09,0.35,1.52,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.85,32.0,X-MC,7.42,79320.0,-4190.0,24216.0,-0.25,-5.02,30.53,23.98,82.0,-0.17,0.56,0.51,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.27,42.0,X-SC,4.21,81395.0,-63765.0,139690.0,-1.02,-43.93,171.62,52.31,136.0,-0.46,0.57,8.95,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.87,35.0,X-MC,2.86,87150.0,-3528.0,18432.0,-1.37,-3.89,21.15,16.44,101.0,-0.19,0.62,15.50,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87308.0,-41362.0,86662.0,0.11,-32.15,99.26,35.20,219.0,-0.48,0.62,0.21,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.65,44.0,X-LC,9.73,279866.0,-66090.0,132796.0,-1.12,-19.10,47.45,19.28,1.0,-0.50,1.98,3.46,X40,ATH,IT
39,INFY,2275.00,-19.24,50.0,X-LC,4.98,314175.0,1287.0,170409.0,0.57,0.41,54.24,54.87,3.0,0.01,2.22,8.73,X40,BTT,IT
41,ITC,452.00,-38.77,42.0,X-LC,1.07,197898.0,-2240.0,23134.0,-0.69,-1.12,11.69,10.44,4.0,-0.10,1.40,4.38,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,6.33,299924.0,-15918.0,128577.0,-0.37,-5.04,42.87,35.67,5.0,-0.12,2.12,8.12,X40N,ATH,FMCG
1,ABB,7934.00,-42.40,35.0,H-LC,5.09,240720.0,-20899.0,140123.0,-4.07,-7.99,58.21,45.57,7.0,-0.15,1.70,2.43,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,32.0,L-SC,27.47,73075.0,-40474.0,80521.0,-2.93,-35.64,110.19,35.27,268.0,-0.50,0.52,78.85,XR,NTT,HOTELS
7,ASIANTILES,137.00,7338.89,62.0,L-SC,12.29,81631.0,-12179.0,88798.0,-2.15,-12.98,108.78,81.67,269.0,-0.14,0.58,58.08,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,49.0,H-SC,8.34,91800.0,-6180.0,27779.0,0.59,-6.31,30.26,22.05,152.0,-0.22,0.65,33.44,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,6.25,123630.0,8832.0,85206.0,-0.47,7.69,68.92,81.92,223.0,0.10,0.87,46.36,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.95,51.0,H-SC,7.95,126101.0,804.0,58120.0,-2.03,0.64,46.09,47.03,141.0,0.01,0.89,57.89,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,75.76,66.0,H-SC,4.07,152308.0,-30467.0,141540.0,1.54,-16.67,92.93,60.77,125.0,-0.22,1.07,37.16,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-31.14,69.0,M-SC,6.43,85750.0,-15409.0,55412.0,1.47,-15.23,64.62,39.54,202.0,-0.28,0.61,20.92,AR,ATH,AUTO
46,KPIGREEN,731.05,5.95,51.0,H-SC,7.95,126101.0,804.0,58120.0,-2.03,0.64,46.09,47.03,141.0,0.01,0.89,57.89,MH,ATH,POWER
62,SAMMAANCAP,326.00,-197.73,70.0,M-SC,34.05,168597.0,19377.0,124795.0,1.10,12.99,74.02,96.62,208.0,0.16,1.19,82.46,XY25,NTT,FINANCE
5,ANGELONE,3033.00,18.88,63.0,X-SC,6.03,210330.0,19324.0,38385.0,3.16,10.12,18.25,30.21,157.0,0.50,1.48,33.11,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.36
2,50,75.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.77
LC    33.13
MC    23.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.99
X40      16.28
X40N     12.46
XY25     11.95
XR        9.88
AR        7.80
OX40N     6.50
X200      1.77
X5K       1.46
SR        1.04
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.22
X-LC    22.44
X-MC    18.76
M-SC    12.58
X-SC     5.54
M-LC     5.16
H-LC     4.55
H-MC     2.33
M-MC     1.71
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.26,-4.68,38.53
IT,12.44,-19.39,83.36
FINANCE,12.22,-9.26,58.78
MISC,7.79,-17.83,77.26
ELECTRICAL,5.70,-11.98,51.67
PAINTS,5.56,-16.97,34.40
BANKS,4.05,-17.95,86.56
INSURANCE,3.90,-0.11,37.43
AUTO,2.73,-48.43,110.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3322150.0,22
XR,1257805.0,13
AR,1183372.0,9
X40,852389.0,12
X40N,703235.0,9
OX40N,564178.0,9
XY25,480328.0,8
SR,274274.0,2
X5K,111446.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3398549.0,24
M-SC,1261061.0,15
X-MC,1218579.0,14
X-LC,1100167.0,13
X-SC,563104.0,6
H-LC,294476.0,3
M-LC,279300.0,4
L-SC,259327.0,3
H-MC,258512.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1192583.0      6
           AR         839391.0      5
           XR         783273.0      7
M-SC       XY24       731621.0      6
X-MC       XY24       574272.0      4
X-LC       X40        467321.0      5
           XY24       313451.0      2
X-SC       X40N       310110.0      4
X-MC       X40        298406.0      6
H-SC       SR         274274.0      2
           OX40N      250908.0      3
X-LC       X40N       234155.0      3
H-LC       AR         221271.0      2
X-MC       XY25       186931.0      2
H-MC       XY24       182946.0      1
L-SC       XR         169319.0      2
X-SC       XY24       166332.0      1
X-MC       X40N       158970.0      2
M-MC       XY24       149634.0      1
M-SC       OX40N      147696.0      4
           XR         134239.0      2
           XY25       124795.0      1
           AR         122710.0      2
M-LC       XR         113487.0      1
           X5K        111446.0      1
L-SC       OX40N       90008.0      1
X-SC       X40         86662.0      1
X-LC       XY25        85240.0      3
H-MC       OX40N       75566.0      1
H-LC       X200        73205.0      1
H-SC       MH          58120.0      1
L-MC       XR          57487.0      1
M-LC       XY25        43056.0      1
L-LC       XY25        40306.0      1
M-LC       XY24        11311.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
